# Sequential Conv + Pool +Flatten + Dense

## 00. Imports

In [1]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import confusion_matrix

In [2]:
import sys
sys.path.append("../")

In [3]:
from src.activation.relu import ReluLayer
from src.layers.pooling import MaxPoolLayer
from src.activation.softmax import SoftmaxLayer
from src.layers.dense import DenseLayer
from src.layers.flatten import FlattenLayer
from src.layers.convolutional import ConvLayer2D
from src.model.sequential import SequentialModel
from src.utils.core import convert_categorical2one_hot, convert_prob2categorical
from src.utils.metrics import calculate_accuracy

## 01. Settings

In [4]:
# number of samples in the train data set
N_TRAIN_SAMPLES = 5000
# number of samples in the test data set
N_TEST_SAMPLES = 500
# number of samples in the validation data set
N_VALID_SAMPLES = 500
# number of classes
N_CLASSES = 10
# image size
IMAGE_SIZE = 28

## 02. Build data set

In [5]:
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
print("trainX shape:", trainX.shape)
print("trainY shape:", trainY.shape)
print("testX shape:", testX.shape)
print("testY shape:", testY.shape)

trainX shape: (60000, 28, 28)
trainY shape: (60000,)
testX shape: (10000, 28, 28)
testY shape: (10000,)


In [6]:
X_train = trainX[:N_TRAIN_SAMPLES, :, :]
y_train = trainY[:N_TRAIN_SAMPLES]

X_test = trainX[N_TRAIN_SAMPLES:N_TRAIN_SAMPLES+N_TEST_SAMPLES, :, :]
y_test = trainY[N_TRAIN_SAMPLES:N_TRAIN_SAMPLES+N_TEST_SAMPLES]

X_valid = testX[:N_VALID_SAMPLES, :, :]
y_valid = testY[:N_VALID_SAMPLES]

**NOTE:** We need to change the data format to the shape supported by my implementation.

In [7]:
X_train = np.moveaxis(X_train, 0, -1) / 255
X_train = np.expand_dims(X_train, axis=2)
y_train = convert_categorical2one_hot(y_train).T
X_test = np.moveaxis(X_test, 0, -1) / 255
X_test = np.expand_dims(X_test, axis=2)
y_test = convert_categorical2one_hot(y_test).T
X_valid = np.moveaxis(X_valid, 0, -1) / 255
X_valid = np.expand_dims(X_valid, axis=2)
y_valid = convert_categorical2one_hot(y_valid).T
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (28, 28, 1, 5000)
y_train shape: (10, 5000)
X_test shape: (28, 28, 1, 500)
y_test shape: (10, 500)
X_valid shape: (28, 28, 1, 500)
y_valid shape: (10, 500)


## 03. Build model

In [8]:
layers = [
    ConvLayer2D.initialize(filters=16, kernel_shape=(3, 3, 1)), # input (28, 28, 1, N) out (26, 26, 16, N)
    ReluLayer(), # input (26, 26, 16, N) out (26, 26, 16, N)
    MaxPoolLayer(pool_size=(2, 2), strides=2), # input (26, 26, 16, N) out (13, 13, 16, N)
    ConvLayer2D.initialize(filters=32, kernel_shape=(5, 5, 16)), # input (13, 13, 16, N) out (9, 9, 32, N)
    ReluLayer(), # input (9, 9, 32, N) out (9, 9, 32, N)
    MaxPoolLayer(pool_size=(3, 3), strides=3), # input (9, 9, 32, N) out (3, 3, 32, N)
    FlattenLayer(), # input (3, 3, 32, N) out (288, N)
    DenseLayer.initialize(input_dim=288, output_dim=256),
    ReluLayer(),
    DenseLayer.initialize(input_dim=256, output_dim=256),
    ReluLayer(),
    DenseLayer.initialize(input_dim=256, output_dim=128),
    ReluLayer(),
    DenseLayer.initialize(input_dim=128, output_dim=64),
    ReluLayer(),
    DenseLayer.initialize(input_dim=64, output_dim=N_CLASSES),
    SoftmaxLayer()
]

model = SequentialModel(layers=layers)

## 04. Train

In [9]:
model.train(
    x_train=X_train, 
    y_train=y_train, 
    x_test=X_test, 
    y_test=y_test, 
    epochs=40, 
    lr=0.005,
    batch_size=512
)

Iteration: 00000 - cost: 2.30212 - accuracy: 0.14800
Iteration: 00001 - cost: 2.27781 - accuracy: 0.14800
Iteration: 00002 - cost: 2.25852 - accuracy: 0.16600
Iteration: 00003 - cost: 2.24184 - accuracy: 0.20200
Iteration: 00004 - cost: 2.22653 - accuracy: 0.24600
Iteration: 00005 - cost: 2.21163 - accuracy: 0.25400
Iteration: 00006 - cost: 2.19663 - accuracy: 0.26200
Iteration: 00007 - cost: 2.18145 - accuracy: 0.26200
Iteration: 00008 - cost: 2.16603 - accuracy: 0.27000
Iteration: 00009 - cost: 2.15047 - accuracy: 0.27400
Iteration: 00010 - cost: 2.13464 - accuracy: 0.28000
Iteration: 00011 - cost: 2.11826 - accuracy: 0.28800
Iteration: 00012 - cost: 2.10124 - accuracy: 0.29800
Iteration: 00013 - cost: 2.08358 - accuracy: 0.31000
Iteration: 00014 - cost: 2.06531 - accuracy: 0.32400
Iteration: 00015 - cost: 2.04652 - accuracy: 0.34200
Iteration: 00016 - cost: 2.02728 - accuracy: 0.35000
Iteration: 00017 - cost: 2.00770 - accuracy: 0.36000
Iteration: 00018 - cost: 1.98777 - accuracy: 0